<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-1-NLP/blob/master/OK_Keras_embeddings_Conv1D_NLP_whiskey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np


In [0]:
#USING Keras embeddings and Conv1D model: https://keras.io/examples/imdb_cnn/
#DATASET: WHISKY RATINGS CLASSIFICATION FROM REVIEWS

In [0]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/bs3537/DS-Unit-4-Sprint-1-NLP/master/module3-document-classification/train.csv')

In [4]:
train.head(1)

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1


In [0]:
train['text'] = train['description'].str.strip('\n')

In [6]:
train.head(1)

,id,description,ratingCategory,text
0,1321,"\nSometimes, when whisky is batched, a few lef...",1,"Sometimes, when whisky is batched, a few lefto..."


In [0]:
train2 = train.drop(columns=['description', 'id'])

In [8]:
train2.head(1)

,ratingCategory,text
0,1,"Sometimes, when whisky is batched, a few lefto..."


In [0]:
train3 = train2.rename(columns={"ratingCategory": "label"})

In [10]:
train3.head(1)

,label,text
0,1,"Sometimes, when whisky is batched, a few lefto..."


In [11]:
from sklearn.model_selection import train_test_split

df_trn, df_val = train_test_split(train3, stratify = train3['label'], test_size = 0.20, random_state=42)

df_trn.shape, df_val.shape

((3269, 2), (818, 2))

In [0]:
#Define train and val datasets and X, Y vectors

In [0]:
X_train = df_trn['text'].values
y_train = df_trn['label'].values
X_val = df_val['text'].values
y_val = df_val['label'].values


In [0]:
#Tokenize using Keras 

In [0]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
tokenizer_obj = Tokenizer()
total_reviews = df_trn['text'].values
tokenizer_obj.fit_on_texts(total_reviews)

In [0]:
#pad sequences

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
max_length = max([len(s.split()) for s in total_reviews])

In [0]:
#define vocabulary size

vocab_size = len(tokenizer_obj.word_index) + 1

In [0]:
#define X_train and X_val tokens

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)

X_val_tokens = tokenizer_obj.texts_to_sequences(X_val)


In [0]:
#Define X_train and X_val pads

X_train_pad = pad_sequences(X_train_tokens, maxlen = max_length, padding='post')

X_val_pad = pad_sequences(X_val_tokens, maxlen = max_length, padding='post')

In [0]:
#Build Keras Model

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, Activation

In [0]:
#The Embedding layer requires the specification of the vocabulary size (vocab_size), the size of the real-valued vector space EMBEDDING_DIM = 100, and the maximum length of input documents max_length

In [0]:
model = Sequential()

In [0]:
#stacking layers on model
model.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length))
model.add(Dropout(0.2))

In [0]:
model.add(Conv1D(filters=250,
                 kernel_size=3,
                 padding='valid',
                 activation='relu',
                 strides=1))

In [0]:
model.add(GlobalMaxPooling1D())

In [0]:
# We add a vanilla hidden layer:
model.add(Dense(units=250))
model.add(Dropout(0.2))
model.add(Activation('relu'))

In [0]:
# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 237, 64)           846592    
_________________________________________________________________
dropout (Dropout)            (None, 237, 64)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 235, 250)          48250     
_________________________________________________________________
global_max_pooling1d (Global (None, 250)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               62750     
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation (Activation)      (None, 250)               0

In [33]:
model.fit(X_train_pad, y_train, batch_size=32, epochs=2, verbose=2, validation_data=(X_val_pad, y_val))

#start with 100 epochs then tune after viewing the losses and accuracy plot
#increasing batch size may increase accuracy

Epoch 1/2
103/103 - 1s - loss: 0.5811 - accuracy: 0.7030 - val_loss: 0.5581 - val_accuracy: 0.7054
Epoch 2/2
103/103 - 1s - loss: 0.5300 - accuracy: 0.7051 - val_loss: 0.5512 - val_accuracy: 0.7152


In [0]:
#Similar accuracy as GRU, but faster.
#Val accuracy decreases after 2 epochs.